## AWS IoT DR - Request device certificates from PCA

First you create a key and a CSR. The CSR will be send to PCA which issues a certificate. These certificates can be used by your devices.

Private, public key and the certificate will be stored on the local file system.

In [ ]:
from OpenSSL import crypto, SSL
from os.path import exists, join
from os import makedirs
from shutil import copy
from time import time, gmtime, localtime, strftime
import boto3
import json
import time

## Shared variables

Import shared variables into this notebook.

In [ ]:
%store -r config
print("config: {}".format(json.dumps(config, indent=4, default=str)))

## Some handy functions

Create a key, a CSR and get the certificate from your private CA.

In [ ]:
def create_csr(pkey, subject, digest="sha256"):
    print("subject: {}".format(subject))
    req = crypto.X509Req()
    subj = req.get_subject()
    
    for i in ['C', 'ST', 'L', 'O', 'OU', 'CN']:
        if i in subject:
            setattr(subj, i, subject[i])

    req.set_pubkey(pkey)
    req.sign(pkey, digest)
    return req


def create_priv_key_and_csr(cert_dir, csr_file, key_file, subject):
    if not exists(cert_dir):
        print("creating directory: {}".format(cert_dir))
        makedirs(cert_dir)
        
    priv_key = crypto.PKey()
    priv_key.generate_key(crypto.TYPE_RSA, 2048)
    #print(crypto.dump_privatekey(crypto.FILETYPE_PEM, priv_key).decode('utf-8'))

    key_file = join(cert_dir, key_file)
    f = open(key_file,"w")
    f.write(crypto.dump_privatekey(crypto.FILETYPE_PEM, priv_key).decode('utf-8'))
    f.close()
    
    csr = create_csr(priv_key, subject)

    csr_file = join(cert_dir, csr_file)
    f= open(csr_file,"w")
    f.write(crypto.dump_certificate_request(crypto.FILETYPE_PEM, csr).decode('utf-8'))
    f.close()
    
    return crypto.dump_certificate_request(crypto.FILETYPE_PEM, csr)


def request_cert_from_pca(subject):
    device_key_file = '{}.device.key.pem'.format(subject['CN'])
    device_csr_file = '{}.device.csr.pem'.format(subject['CN'])
    device_cert_file = '{}.device.cert.pem'.format(subject['CN'])
    device_root_cert_file = '{}.device.root.cert.pem'.format(subject['CN'])
    
    device_csr = create_priv_key_and_csr(config['PCA_directory'], 
                                         device_csr_file, 
                                         device_key_file, 
                                         subject)
    print("device_csr: {}".format(device_csr))

    idempotency_token = '{}_id_token'.format(subject['CN'])
    response = c_acm_pca.issue_certificate(
        CertificateAuthorityArn = pca_arn,
        Csr = device_csr,
        SigningAlgorithm = 'SHA256WITHRSA',
        Validity= {
            'Value': 365,
            'Type': 'DAYS'
        },
        IdempotencyToken = idempotency_token
    )

    print("response: {}".format(response))

    certificate_arn = response['CertificateArn']
    print("certificate_arn: {}".format(certificate_arn))
   
    waiter = c_acm_pca.get_waiter('certificate_issued')
    try:
        waiter.wait(
            CertificateAuthorityArn=pca_arn,
            CertificateArn=certificate_arn
        )
    except botocore.exceptions.WaiterError as e:
        raise Exception("waiter: {}".format(e))
    
    response = c_acm_pca.get_certificate(
        CertificateAuthorityArn = pca_arn,
        CertificateArn = certificate_arn
    )
    print("response: {}".format(response))
    device_cert = response['Certificate']
    print("device_cert: {}".format(device_cert))

    file_device_crt = join(config['PCA_directory'], device_cert_file)
    f = open(file_device_crt,"w")
    f.write(device_cert)
    f.close()
    
    file_root_device_crt = join(config['PCA_directory'], device_root_cert_file)
    f = open(file_root_device_crt,"w")
    f.write(device_cert)
    f.write("\n")
    f.write(pca_certificate)
    f.close()
    
    print("device_key_file: {}".format(device_key_file))
    print("device_csr_file: {}".format(device_csr_file))
    print("device_cert_file: {}".format(device_cert_file))
    print("device_root_cert_file: {}".format(device_root_cert_file))

## Boto3 client

In [ ]:
c_acm_pca = boto3.client('acm-pca', region_name = config['aws_region_pca'])

## PCA certificate and ARN
Get the CA certificate for the private CA as well as it's arn. The arn is required to issue certificates and for the private CA certificate will be stored together with the device certificate.

In [ ]:
response = c_acm_pca.list_certificate_authorities(MaxResults=50)

for ca in response['CertificateAuthorities']:
    if ca['CertificateAuthorityConfiguration']['Subject']['CommonName'] == config['Sub_CN']:
        pca_arn = ca['Arn']
        break

print("pca_arn: {}".format(pca_arn))

response = c_acm_pca.get_certificate_authority_certificate(
    CertificateAuthorityArn = pca_arn
)
#print("response: {}".format(json.dumps(response, indent=4, default=str)))
pca_certificate = response['Certificate']
print("pca_certificate:\n{}".format(pca_certificate))

## Request device certificate
Set the common name (CN) in your device certificate to be your thing or device name. Provide a thing name in the variable `thing_name`. 

The Just-in-Time Registration process will extract the CN from the certificate and use it as thing name.

In [ ]:
thing_name = "dr-pca-04"
request_cert_from_pca({"CN": thing_name})